In [28]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Reading data
accepted = pd.read_csv('data/LC/Lending_Club_Accepted_2014_2018.csv')
rejected = pd.read_csv('data/LC/Lending_Club_Rejected_2014_2018.csv')

stripped_accepted = accepted[['loan_amnt', 'issue_d', 'title', 'dti', 'zip_code', 'addr_state', 'emp_length', 'policy_code']]
stripped_accepted['accepted'] = 1
rejected.rename(columns={'Amount_Requested': 'loan_amnt',
                         'Application_Date': 'date',
                         'Loan_Title': 'title',
                         'Risk_Score': 'risk_score',
                         'Debt_to_Income_Ratio': 'dti',
                         'Zip_Code': 'zip_code',
                         'State': 'state',
                         'Employment_Length': 'emp_length',
                         'Policy_Code': 'policy_code'}, inplace=True)
rejected.drop(columns='risk_score', inplace=True)
rejected['accepted'] = 0
stripped_accepted.rename(columns={'issue_d': 'date',
                                  'addr_state': 'state'}, inplace=True)
full_data = pd.concat([stripped_accepted, rejected])
full_data.head()

,loan_amnt,date,title,dti,zip_code,state,emp_length,policy_code,accepted
0,3600.0,Dec-2015,Debt consolidation,5.91,190xx,PA,10+ years,1.0,1
1,24700.0,Dec-2015,Business,16.06,577xx,SD,10+ years,1.0,1
2,20000.0,Dec-2015,NaN,10.78,605xx,IL,10+ years,1.0,1
3,35000.0,Dec-2015,Debt consolidation,17.06,076xx,NJ,10+ years,1.0,1
4,10400.0,Dec-2015,Major purchase,25.37,174xx,PA,3 years,1.0,1


In [6]:
# Cleaning loan titles
loan_titles = ['Debt Consolidation', 'Credit Card', 'Home', 'Car', 'Medical', 'Business', 'Other']
def clean_titles(full_data):
    full_data['title'] = full_data['title'].fillna('')
    full_data['title'] = full_data['title'].str.lower()
    full_data.loc[full_data['title'].str.contains('credit'), 'title'] = 'Credit Card'
    full_data.loc[full_data['title'].str.contains('cc'), 'title'] = 'Credit Card'
    full_data.loc[full_data['title'].str.contains('debt'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('consolidation'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('consolidate'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('payoff'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('pay off'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('refinance'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('home'), 'title'] = 'Home'
    full_data.loc[full_data['title'].str.contains('house'), 'title'] = 'Home'
    full_data.loc[full_data['title'].str.contains('moving'), 'title'] = 'Home'
    full_data.loc[full_data['title'].str.contains('car'), 'title'] = 'Car'
    full_data.loc[full_data['title'].str.contains('medical'), 'title'] = 'Medical'
    full_data.loc[full_data['title'].str.contains('business'), 'title'] = 'Business'
    full_data.loc[~full_data['title'].isin(loan_titles), 'title'] = 'Other'
    return full_data

In [7]:
full_data = clean_titles(full_data)

In [8]:
urban_rural_states = pd.read_csv('data/census_data/2010_urban_rural_states.csv', skiprows=1)
state_abbr = pd.read_csv('data/census_data/state_abbr.csv')
urban_rural_states['total_pop'] = urban_rural_states['Total!!Rural'] + urban_rural_states['Total!!Urban']
urban_rural_states['rural_prop'] = urban_rural_states['Total!!Rural'] / urban_rural_states['total_pop']
urban_rural_states = urban_rural_states.merge(state_abbr, left_on='Geographic Area Name', right_on='State')
urban_rural_states = urban_rural_states.rename(columns={'Code': 'state'})
urban_rural_states = urban_rural_states[['state', 'rural_prop', 'total_pop']]
urban_rural_states.head(10)

,state,rural_prop,total_pop
0,AL,0.416524,2171853
1,AK,0.403881,306967
2,AZ,0.115668,2844526
3,AR,0.452568,1316299
4,CA,0.063259,13680081
5,CO,0.168969,2212898
6,CT,0.120540,1487891
7,DE,0.164515,405885
8,DC,0.000000,296719
9,FL,0.088944,8989580


In [9]:
relevant_cols = ['title', 'emp_title', 'emp_length', 'fico_range_low', 'fico_range_high',
                 'zip_code', 'addr_state', 'loan_amnt', 'grade', 'dti', 'term', 'int_rate',
                 'home_ownership', 'issue_d']
filtered_accepted = accepted[relevant_cols]
filtered_accepted['fico'] = (filtered_accepted['fico_range_low'] + filtered_accepted['fico_range_high']) / 2
filtered_accepted.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)
filtered_accepted = clean_titles(filtered_accepted)
filtered_accepted = filtered_accepted.rename(columns={'addr_state': 'state'})
filtered_accepted = filtered_accepted.merge(urban_rural_states, on='state')
filtered_accepted['issue_d'] = pd.to_datetime(filtered_accepted['issue_d']).dt.strftime('%m-%Y')
filtered_accepted.head()

,title,emp_title,emp_length,zip_code,state,loan_amnt,grade,dti,term,int_rate,home_ownership,issue_d,fico,rural_prop,total_pop
0,Debt Consolidation,leadman,10+ years,190xx,PA,3600.0,C,5.91,36 months,13.99,MORTGAGE,12-2015,677.0,0.224589,5567315
1,Other,Contract Specialist,3 years,174xx,PA,10400.0,F,25.37,60 months,22.45,MORTGAGE,12-2015,697.0,0.224589,5567315
2,Credit Card,SERVICE MANAGER,6 years,160xx,PA,10000.0,A,13.07,36 months,6.49,RENT,12-2015,687.0,0.224589,5567315
3,Debt Consolidation,Mechanic,10+ years,172xx,PA,18000.0,A,15.56,36 months,7.91,MORTGAGE,12-2015,742.0,0.224589,5567315
4,Debt Consolidation,Graduate Student,1 year,191xx,PA,3600.0,C,26.45,36 months,13.44,RENT,12-2015,712.0,0.224589,5567315


In [10]:
pre_2016 = filtered_accepted[filtered_accepted['issue_d'] < '05-2016']
post_2016 = filtered_accepted[filtered_accepted['issue_d'] > '05-2016']

In [11]:
full_data['date'] = pd.to_datetime(full_data['date']).dt.strftime('%m-%Y')
full_data_pre_2016 = full_data[full_data['date'] < '05-2016']
full_data_post_2016 = full_data[full_data['date'] > '05-2016']

In [18]:
filtered_accepted = filtered_accepted[~filtered_accepted['dti'].isna()]

In [ ]:
filtered_accepted = filtered_accepted.join(pd.get_dummies(filtered_accepted['grade'], drop_first=True))

In [41]:
# int_rate ~ grade, fico, dti
X, y = sm.add_constant(filtered_accepted[['B', 'C', 'D', 'E', 'F', 'G', 'fico', 'dti']]), filtered_accepted['int_rate']
ols = sm.OLS(y, X)
ols_results = ols.fit()
print(ols_results.summary())

                            OLS Regression Results                            
Dep. Variable:               int_rate   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                 2.755e+06
Date:                Fri, 22 Jul 2022   Prob (F-statistic):               0.00
Time:                        15:50:08   Log-Likelihood:            -3.5805e+06
No. Observations:             2028241   AIC:                         7.161e+06
Df Residuals:                 2028232   BIC:                         7.161e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.3610      0.025    375.877      0.0

In [42]:
filtered_accepted['predicted_int_rate'] = ols_results.predict(X)
filtered_accepted['int_rate_residual'] = filtered_accepted['int_rate'] - filtered_accepted['predicted_int_rate']
plot_df = filtered_accepted.groupby('state').mean()['int_rate_residual'].reset_index()
plot_df = plot_df[~plot_df['state'].isin(['IA'])]

fig = px.choropleth(plot_df,
                    locations='state', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='int_rate_residual',
                    color_continuous_scale="PuBu")
fig.show()

In [39]:
# int_rate ~ grade, fico, dti, rural_prop
X, y = sm.add_constant(filtered_accepted[['B', 'C', 'D', 'E', 'F', 'G', 'fico', 'dti', 'rural_prop']]), filtered_accepted['int_rate']
ols = sm.OLS(y, X)
ols_results = ols.fit()
print(ols_results.summary())

                            OLS Regression Results                            
Dep. Variable:               int_rate   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                 2.449e+06
Date:                Fri, 22 Jul 2022   Prob (F-statistic):               0.00
Time:                        15:47:14   Log-Likelihood:            -3.5804e+06
No. Observations:             2028241   AIC:                         7.161e+06
Df Residuals:                 2028231   BIC:                         7.161e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.3663      0.025    375.677      0.0